In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
#import seaborn as sns
#sns.set_style("whitegrid")
#sns.set_context("poster")

In [2]:
import requests
import time
import json

## Automated Code: 1960s -past

In [16]:
%%time
count = 2018 #enter new count starting number
year = '1956' #enter year
startmonths = ['01', '03', '05', '07', '09', '11']
endmonths = ['02', '04', '06', '08', '10', '12']
startdays = ['01', '01', '01', '01', '01', '01']
enddays = ['27', '30', '30', '31', '31', '31']

pcount = 0

for d in range(6):
    sdate = year + startmonths[d] + startdays[d]
    edate = year + endmonths[d] + enddays[d]

    docs=[]
    #get 1st page and number of documents
    url1 = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page=1&begin_date={}&end_date={}&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790".format(sdate, edate) 
    lpage = requests.get(url1).json()['response']['meta']['hits']
    
    #calculate number of pages for call; if there are no hits skip to end
    if lpage is not 0:
        numpages = int(lpage/10 + 2) 
        pcount += numpages
    
        #get json files for first page
        pagedoc1 = requests.get(url1).json()['response']['docs']
        for j in range(0,len(pagedoc1)):
            docs.append(pagedoc1[j])    
    
        #get json dictionaries for rest of the pages
        for i in range(2, numpages): 
            url = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page={}&begin_date={}&end_date={}&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790".format(i, sdate, edate)
            pagedocs = requests.get(url).json()['response']['docs']
            time.sleep(1)
        
            for j in range(0,len(pagedocs)):
                docs.append(pagedocs[j])

    #pull information from json file into dictionary        
        docsinfo = []
        for d in docs:
            obs = {}
            obs['id'] = d['_id']
            obs['type'] = d['type_of_material']
            obs['doctype'] = d['document_type']
            obs['date'] = d['pub_date']
            obs['news_desk'] = d['news_desk']
            obs['section'] = d['section_name']
            obs['subsection'] = d['subsection_name']
            obs['abstract'] = d['abstract']
            obs['paragraph'] = d['lead_paragraph']
        
            if d['headline'].get('main') is not None:
                obs['headline'] = d['headline']['main']
            elif d['headline'].get('name') is not None:
                obs['headline'] = d['headline']['name']
            else:
                obs['headline'] = ' '
    
            if obs['date'] is not None:
                obs['date'] = obs['date'][0:10]
    
            #Take out abstracts and lead paragraphs with none to join text
            if obs['abstract'] is None:
                a = ' '
            else: 
                a = obs['abstract']
            if obs['paragraph'] == 'TK TK TK' or obs['paragraph'] is None:
                p = ' '
            else:
                p = obs['paragraph']
    
            text = [obs['headline'], p, a]
            obs['text'] = " ".join(text)
    
            docsinfo.append(obs)

    #create dataframe from dictionary, make date column date type, and store in csv
        docsdf = pd.DataFrame(docsinfo)
        docsdf['date'] = pd.to_datetime(docsdf['date'])
        docsdf.to_csv("data/docsdf-{}.csv".format(count), encoding = 'utf-8') 

    print(count, pcount) 
    count += 1 
    

2018 84
2019 168
2020 254
2021 331
2022 412
2023 480
Wall time: 12min 17s


In [4]:

docs=[]
ns = 1 #number of pages

#get first 100 pages of json dictionaries
for i in range(1,101): 
    url = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page=%d&begin_date=20121211&end_date=20121220&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790" % i
    page = requests.get(url)
    pagedocs = page.json()['response']['docs']
    time.sleep(1)
    
    #append each separate dictionary from a page to docs
    for j in range(0,len(pagedocs)):
        docs.append(pagedocs[j])

In [102]:
fd=open("data/nytdocs-19.json","w")
json.dump(docs, fd)
fd.close()

In [6]:
docsinfo = []
for d in docs:
    obs = {}
    obs['id'] = d['_id']
    obs['type'] = d['type_of_material']
    obs['doctype'] = d['document_type']
    obs['date'] = d['pub_date']
    obs['news_desk'] = d['news_desk']
    obs['section'] = d['section_name']
    obs['subsection'] = d['subsection_name']
    obs['abstract'] = d['abstract']
    obs['paragraph'] = d['lead_paragraph']
    
    if d['headline'].get('main') is not None:
        obs['headline'] = d['headline']['main']
    elif d['headline'].get('name') is not None:
        obs['headline'] = d['headline']['name']
    else:
        obs['headline'] = ' '
    
    if obs['date'] is not None:
        obs['date'] = obs['date'][0:10]
    
    #Take out abstracts and lead paragraphs with none to join text
    if obs['abstract'] is None:
        a = ' '
    else: 
        a = obs['abstract']
    if obs['paragraph'] == 'TK TK TK' or obs['paragraph'] is None:
        p = ' '
    else:
        p = obs['paragraph']
    
    text = [obs['headline'], p, a]
    obs['text'] = " ".join(text)
    
    docsinfo.append(obs)

In [7]:
docsdf = pd.DataFrame(docsinfo)
docsdf.head(2)

""


In [15]:
docsdf.shape

(227, 11)

In [16]:
docsdf['date'] = pd.to_datetime(docsdf['date'])

In [17]:
docsdf.to_csv("data/docsdf-814.csv", encoding = 'utf-8')

## Automated code

In [12]:
%%time
count = 1964 #enter new count starting number
year = '1960' #enter year
months = ['01', '01', '01', '02', '02', '02', '03', '03', '03', '04', '04', '04', '05', '05', '05', '06', '06', '06', '07', '07', '07', '08', '08', '08', '09', '09', '09', '10', '10', '10', '11', '11', '11', '12', '12', '12']
startdays = ['01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21']
enddays = ['10', '20', '31', '10', '20', '27', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '30', '10', '20', '31']

pcount = 0

for d in range(36):
    sdate = year + months[d] + startdays[d]
    edate = year + months[d] + enddays[d]

    docs=[]
    #get 1st page and number of documents
    url1 = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page=1&begin_date={}&end_date={}&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790".format(sdate, edate) 
    lpage = requests.get(url1).json()['response']['meta']['hits']
    
    #calculate number of pages for call; if there are no hits skip to end
    if lpage is not 0:
        numpages = int(lpage/10 + 2) 
        pcount += numpages
    
        #get json files for first page
        pagedoc1 = requests.get(url1).json()['response']['docs']
        for j in range(0,len(pagedoc1)):
            docs.append(pagedoc1[j])    
    
        #get json dictionaries for rest of the pages
        for i in range(2, numpages): 
            url = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page={}&begin_date={}&end_date={}&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790".format(i, sdate, edate)
            pagedocs = requests.get(url).json()['response']['docs']
            time.sleep(1)
        
            for j in range(0,len(pagedocs)):
                docs.append(pagedocs[j])

    #pull information from json file into dictionary        
        docsinfo = []
        for d in docs:
            obs = {}
            obs['id'] = d['_id']
            obs['type'] = d['type_of_material']
            obs['doctype'] = d['document_type']
            obs['date'] = d['pub_date']
            obs['news_desk'] = d['news_desk']
            obs['section'] = d['section_name']
            obs['subsection'] = d['subsection_name']
            obs['abstract'] = d['abstract']
            obs['paragraph'] = d['lead_paragraph']
        
            if d['headline'].get('main') is not None:
                obs['headline'] = d['headline']['main']
            elif d['headline'].get('name') is not None:
                obs['headline'] = d['headline']['name']
            else:
                obs['headline'] = ' '
    
            if obs['date'] is not None:
                obs['date'] = obs['date'][0:10]
    
            #Take out abstracts and lead paragraphs with none to join text
            if obs['abstract'] is None:
                a = ' '
            else: 
                a = obs['abstract']
            if obs['paragraph'] == 'TK TK TK' or obs['paragraph'] is None:
                p = ' '
            else:
                p = obs['paragraph']
    
            text = [obs['headline'], p, a]
            obs['text'] = " ".join(text)
    
            docsinfo.append(obs)

    #create dataframe from dictionary, make date column date type, and store in csv
        docsdf = pd.DataFrame(docsinfo)
        docsdf['date'] = pd.to_datetime(docsdf['date'])
        docsdf.to_csv("data/docsdf-{}.csv".format(count), encoding = 'utf-8') 

    print(count, pcount) 
    count += 1 
    

1964 17
1965 32
1966 46
1967 58
1968 71
1969 81
1970 92
1971 105
1972 121
1973 138
1974 149
1975 162
1976 177
1977 192
1978 205
1979 221
1980 237
1981 249
1982 263
1983 276
1984 293
1985 305
1986 316
1987 333
1988 346
1989 359
1990 370
1991 382
1992 393
1993 408
1994 422
1995 437
1996 448
1997 460
1998 474
1999 485
Wall time: 11min 3s


In [30]:
#testing when key error occurs
for d in docs:
    if d['headline'].get('main') is None:
        print(d['headline'])

{'content_kicker': 'Sports Briefing', 'sub': 'TENNIS ;'}


In [10]:
##To test for page results

docs = []
url1 = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page=1&begin_date=19810711&end_date=19810720&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790" 
lpage = requests.get(url1).json()['response']['meta']['hits']

pagedoc1 = requests.get(url1).json()['response']['docs']
for j in range(0,len(pagedocs)):
    docs.append(pagedoc1[j])

for i in range(2, int(lpage/10 + 2)): 
    url = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page=%d&begin_date=19810711&end_date=19810720&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790" % i
    pagedocs = requests.get(url).json()['response']['docs']
    time.sleep(1)
    print(i)
    
    #append each separate dictionary from a page to docs
    for j in range(0,len(pagedocs)):
        docs.append(pagedocs[j])
    

ValueError: No JSON object could be decoded

In [12]:
print lpage

152


## Other stuff

In [31]:
docsdf.dtypes

abstract              object
date          datetime64[ns]
doctype               object
headline              object
id                    object
news_desk             object
paragraph             object
section               object
subsection            object
text                  object
type                  object
dtype: object

In [ ]:
#with open("nytdocs100.json") as json_file:
#    docs = json.load(json_file)

Make date column a date type

http://stackoverflow.com/questions/9504356/convert-string-into-date-type-on-python

http://stackoverflow.com/questions/16852911/how-do-i-convert-dates-in-a-pandas-data-frame-to-a-date-data-type  ***

In [23]:
docsdf.to_csv("docsdf100-2.csv")  #, encoding = 'utf-8')

In [9]:
docsdf = pd.read_csv("docsdf100-2.csv", encoding = "ISO-8859-1")
docsdf.head()

#http://stackoverflow.com/questions/18171739/unicodedecodeerror-win-reading-csv-file-in-pandas-with-python
#http://stackoverflow.com/questions/24616678/unicodedecodeerror-in-python-when-reading-a-file-how-to-ignore-the-error-and-ju

,Unnamed: 0,abstract,date,doctype,headline,id,news_desk,paragraph,section,subsection,text,type
0,0,NaN,2015-11-03,article,"Know Your Risks, but Meat Still Isnt the Enemy",56374e0a798810408f299696,Upshot,The link between processed meat and cancer is ...,The Upshot,NaN,"Know Your Risks, but Meat Still Isnt the Enem...",News
1,1,Study in journal The Lancet concludes Chinese ...,2015-10-13,article,Study Shows Spread of Cigarettes in China,561bc3a67988101d11e0da9e,Science,Chinese men smoke one-third of all the worlds...,Health,NaN,Study Shows Spread of Cigarettes in China Chin...,News
2,2,Food and Drug Administration approves Narcan n...,2015-11-19,article,F.D.A. Approves a Nasal Spray to Combat Opioid...,564d433579881039a958bb07,National,A nasal spray designed to reverse opioid overd...,U.S.,NaN,F.D.A. Approves a Nasal Spray to Combat Opioid...,Brief
3,3,NaN,2015-11-17,article,How to Decrease Prices for an Expensive Class ...,5649c243798810095961385d,Upshot,Two professors suggest requiring biologic manu...,The Upshot,NaN,How to Decrease Prices for an Expensive Class ...,News
4,4,NaN,2015-10-07,article,Ragout With Farfalle Gets a Touch of Green,56143f92798810321b0fac91,Dining,The sweet taste and texture of broccoli leaves...,Food,NaN,Ragout With Farfalle Gets a Touch of Green The...,News


In [115]:
#solving keyerror issue for headline

ex = {'main': 'words', 'kicker': 'stuff'}
if ex.get('name') is None:
    print('ok')

#http://stackoverflow.com/questions/6130768/return-none-if-dictionary-key-is-not-available

ok
